In [16]:
import gzip
import json
import os
import numpy as np
file_path = '/home/work/afs_input/wangzixuan17/Bench2Drive/Accident_Town12_Route1744_Weather22/anno'
file = os.path.join(file_path, '00123.json.gz')
# 读取压缩的 JSON 文件
with gzip.open(file, 'rt', encoding='utf-8') as f:
    data = json.load(f)
# data = np.load(file)
# 使用 data 变量
print(data['theta'])


4.736570835113525


In [2]:
import os
import tarfile
from multiprocessing import Pool, cpu_count
from tqdm import tqdm
from functools import partial

# ======================== #
# 设置路径
# ======================== #
tar_dir = '/home/work/afs_input/wangzixuan17/Bench2Drive'         # 存放 .tar.gz 文件的目录
output_dir = '/data/bench2drive'        # 提取后保存的根目录
extract_folders = ['anno', 'camera/rgb_front']

# === 提取函数 ===
def extract_selected(tar_path, output_dir, extract_folders):
    try:
        # 提取文件名（无扩展名）
        base_name = os.path.splitext(os.path.splitext(os.path.basename(tar_path))[0])[0]
        output_subdir = os.path.join(output_dir, base_name)

        with tarfile.open(tar_path, 'r:gz') as tar:
            for member in tar.getmembers():
                # member.name 示例：AccidentTwoWays_Town05_Route25864_Weather12/camera/rgb_front/0001.jpg
                parts = member.name.split('/', 1)
                if len(parts) < 2:
                    continue
                relative_path = parts[1]  # 去掉包裹目录，如 camera/rgb_front/0001.jpg

                # 严格限制仅提取 camera/rgb_front/ 和 anno/
                if not any(relative_path.startswith(folder + '/') for folder in extract_folders):
                    continue

                # 构建目标提取路径
                extract_path = os.path.join(output_subdir, relative_path)
                os.makedirs(os.path.dirname(extract_path), exist_ok=True)

                # 仅提取文件（跳过目录等）
                if member.isfile():
                    with tar.extractfile(member) as src, open(extract_path, 'wb') as dst:
                        dst.write(src.read())
        return True
    except Exception as e:
        print(f"[ERROR] Failed to extract {tar_path}: {e}")
        return False

# === 多进程主程序 ===
def main():
    tar_files = [os.path.join(tar_dir, f) for f in os.listdir(tar_dir) if f.endswith('.tar.gz')]
    print(f"Found {len(tar_files)} tar.gz files.")
    
    with Pool(processes=cpu_count()) as pool:
        results = list(tqdm(
            pool.imap_unordered(
                partial(extract_selected, output_dir=output_dir, extract_folders=extract_folders),
                tar_files
            ),
            total=len(tar_files)
        ))

    print(f"Done. Success count: {sum(results)} / {len(tar_files)}")

if __name__ == '__main__':
    main()

Found 13638 tar.gz files.


  0%|          | 0/13638 [00:24<?, ?it/s]


KeyboardInterrupt: 

In [8]:
import os
import numpy as np
import gzip
import json
from tqdm import tqdm
base_path = '/data/bench2drive'
# seq = 'Accident_Town12_Route1744_Weather22'

def main():
    seq_name = sorted(os.listdir(base_path))
    print(seq_name)
    fflushed
    for seq in tqdm(seq_name):
        os.makedirs(os.path.join(base_path, seq, 'decision'),exist_ok=True)
        anno_path = os.path.join(base_path, seq, 'anno')
        json_name = sorted(os.listdir(anno_path))
        for json_f in json_name:
            json_path = os.path.join(anno_path, json_f)
            frame = {}
            with gzip.open(json_path, 'rb') as f:
                data = json.load(f)
                frame['ego_x'] = data['x']
                frame['ego_y'] = data['y']
                frame['ego_yaw'] = data['theta']
            txt_path = os.path.join(os.path.dirname(anno_path),'decision')
            txtx =os.path.join(txt_path,json_f.split('.')[0]+'.json')
            with open(txtx,'w') as f:
                f.write(json.dumps(frame))

if __name__ == "__main__":
    main()

100%|██████████| 9217/9217 [50:56<00:00,  3.02it/s]  


In [3]:
import os
import json
import math
from tqdm import tqdm

def load_decision_jsons(decision_dir):
    files = sorted(f for f in os.listdir(decision_dir) if f.endswith('.json'))
    files.sort()  # 确保顺序
    data = []
    for f in files:
        with open(os.path.join(decision_dir, f)) as jf:
            content = json.load(jf)
        data.append((f, content))
    return data

def global_to_local(x, y, origin_x, origin_y, origin_yaw):
    dx = x - origin_x
    dy = y - origin_y
    cos_yaw = math.cos(-origin_yaw)
    sin_yaw = math.sin(-origin_yaw)
    x_local = dx * cos_yaw - dy * sin_yaw
    y_local = dx * sin_yaw + dy * cos_yaw
    return x_local, y_local

def process_seq_folder(seq_path):
    decision_dir = os.path.join(seq_path, "decision")
    waypoint_dir = os.path.join(seq_path, "waypoints")
    os.makedirs(waypoint_dir, exist_ok=True)

    decision_data = load_decision_jsons(decision_dir)
    total = len(decision_data)

    for i, (fname, current) in enumerate(decision_data):
        ego_x0 = current["ego_x"]
        ego_y0 = current["ego_y"]
        yaw0 = current["ego_yaw"]

        waypoints = []
        for j in range(1, 9):  # 获取未来5帧
            if i + j >= total:
                break  # 超出边界
            _, future = decision_data[i + j]
            fx, fy = future["ego_x"], future["ego_y"]
            lx, ly = global_to_local(fx, fy, ego_x0, ego_y0, yaw0)
            waypoints.append({"x": lx, "y": ly})

        output = {
            "waypoints": waypoints,
        }

        with open(os.path.join(waypoint_dir, fname), 'w') as wf:
            json.dump(output, wf, indent=2)

def main(root_dir):
    for seq_name in tqdm(os.listdir(root_dir)):
    # seq_name = 'Accident_Town12_Route1744_Weather22'
        seq_path = os.path.join(root_dir, seq_name)
        if os.path.isdir(seq_path): #and "seq" in seq_name:
            # print(f"Processing {seq_name}")
            process_seq_folder(seq_path)

if __name__ == "__main__":
    # 修改为你的根目录路径
    root_directory = "/data/bench2drive"
    main(root_directory)


100%|██████████| 9217/9217 [05:17<00:00, 29.00it/s]
